# 协程
微线程。

子程序（函数）在所有语言中都是层级调用，通过栈实现，一个线程就是执行一个子程序。

协程在执行过程中，子程序内部可中断，然后转而执行别的子程序，在后面返回接着执行，不是函数调用。

python中的协程支持，通过generator实现。

python中的yield不仅可以返回一个值，还可以接收调用者发出的参数

## 生产和消费例子
传统的生产者-消费者模型是一个线程写信息，一个线程取消息，通过锁机制控制队列和等待，但一不小心可能死锁

改用协程，生产者生产消息后，直接通过yield跳转到消费者开始执行，完毕后，切换会生产者，效率提高

In [2]:
#消费者
def consumer():
    r=''
    while True:
        n=yield r   #首次执行到此停止，返回空字符串r，等待外部发送数据
        
        #为None或空时
        if not n :
            return

        #n非空时继续执行    
        print('[Consumer]Consuming %s...' % n)
        r = '200 OK'

def produce(c):
    #启动生成器
    c.send(None)
    n=0

    while n<5:
        n=n+1
        print('[Producer]Producing %s...' % n)

        #n=1执行到此，发送数据
        r=c.send(n)
        print('[PRODUCER]Consumer return: %s' % r)
    
    c.close()

c=consumer()    #创建生成器对象，但内部代码未执行
produce(c)


[Producer]Producing 1...
[Consumer]Consuming 1...
[PRODUCER]Consumer return: 200 OK
[Producer]Producing 2...
[Consumer]Consuming 2...
[PRODUCER]Consumer return: 200 OK
[Producer]Producing 3...
[Consumer]Consuming 3...
[PRODUCER]Consumer return: 200 OK
[Producer]Producing 4...
[Consumer]Consuming 4...
[PRODUCER]Consumer return: 200 OK
[Producer]Producing 5...
[Consumer]Consuming 5...
[PRODUCER]Consumer return: 200 OK


# asyncio
python 3.4引入的标准库，内置了对异步IO的支持

asyncio的编程模型是一个消息循环，模块内部实现了eventloop，将需要执行的协程扔到eventloop中执行，实现了异步io

asyncio提供的@asyncio.coroutine可以把一个generator标记为coroutine类型，然后在coroutine内部用yield from调用另一个coroutine实现异步操作

In [7]:
# Hello World例子
import asyncio

async def hello():
    print("Hello world!")

    #异步调用asyncio.sleep(1):
    await asyncio.sleep(1)
    print("Hello again!")

#asyncio.run()设置作为程序的主入口点
await hello()   #jupyter交互环境中已经启动了事件循环

Hello world!
Hello again!


In [ ]:
import threading

#并发执行特点体现
async def hello(name):
    #打印name和当前线程
    print("Hello %s! (%s)" %(name,threading.current_thread))

    #在中断期间不是停止，而是去执行其他异步函数
    await asyncio.sleep(1)

    print("Hello %s again! (%s)" %(name,threading.current_thread))

    return name

#使用gather同时调度多个async函数
async def main():
    L=await asyncio.gather(hello("Bob"),hello("Ash"))
    print(L)

await main()

#打印结果可知，所有内容都是由一个线程执行

Hello Bob! (<function current_thread at 0x000001F3156214E0>)
Hello Ash! (<function current_thread at 0x000001F3156214E0>)
Hello Bob again! (<function current_thread at 0x000001F3156214E0>)
Hello Ash again! (<function current_thread at 0x000001F3156214E0>)
['Bob', 'Ash']


In [ ]:
#将上例中的sleep换成真正的IO操作，则多个并发的IO操作实际可由同一个线程并发执行
import asyncio

#异步抓取函数
async def wget(host):
    print(f"wget {host}...")

    #创建到主机80端口的异步连接
    reader,writer = await asyncio.open_connection(host,80)

    #发送http请求：
    header=f"GET / HTTP/1.0\r\nHost: {host}\r\n\r\n"

    writer.write(header.encode("utf-8"))
    await writer.drain()    #等待数据发送完成

    #读取处理响应头
    while True:
        #异步读取一行
        line = await reader.readline()
        #遇到空行表示头结束
        if line == b"\r\n":
            break
        print("%s header > %s" % (host,line.decode("utf-8").rstrip()))

    #关闭连接
    writer.close()
    await writer.wait_closed()
    print(f"Done {host}")

#并发执行三个抓取任务
async def main():
        await asyncio.gather(wget("www.sina.com.cn"),wget("www.sohu.com"),wget("www.baidu.com"))

await(main())

wget www.sina.com.cn...
wget www.sohu.com...
wget www.baidu.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Fri, 15 Aug 2025 06:36:13 GMT
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Content-Length: 138
www.sina.com.cn header > Connection: close
www.sina.com.cn header > Location: https://www.sina.com.cn/
www.sina.com.cn header > X-Via-CDN: f=sinaedge,s=cmcc.hangzhou.union.73.nb.sinaedge.com,c=39.174.232.254;
Done www.sina.com.cn
www.sohu.com header > HTTP/1.1 302 Found
www.sohu.com header > Location: https://www.sohu.com/
www.sohu.com header > Content-Length: 0
www.sohu.com header > X-NWS-LOG-UUID: 16316417526576983706
www.sohu.com header > Connection: close
www.sohu.com header > Server: D0
www.sohu.com header > Date: Fri, 15 Aug 2025 06:36:13 GMT
www.sohu.com header > X-Cache-Lookup: Return Directly
Done www.sohu.com
www.baidu.com header > HTTP/1.0 200 OK
www.baidu.com hea